In [1]:
import numpy as np
np.random.seed(0)

# Log plot scale
def xlog_scale(log_x_max, scale, log_base= 10): 
    '''Logaritmic scale up to log_alpha_max'''
    bd_block = np.arange(0, log_base**2, log_base) + log_base
    bd_block = bd_block[0:-1]
    xlog = np.tile(bd_block, log_x_max)
    xlog[(log_base-1) : 2*(log_base-1)] = log_base*xlog[(log_base-1) : 2*(log_base-1)]
    for j in range(1, log_x_max - 1):
        xlog[(j+1)*(log_base-1) : (j+2)*(log_base-1)] = log_base*xlog[  j*(log_base-1) :  (j+1)*(log_base-1)  ]
    xlog = np.insert(xlog, 0,  np.arange(1,log_base), axis=0)
    xlog = np.insert(xlog, len(xlog),log_base**(log_x_max+1), axis=0)
    jlog = (xlog*scale).astype(int)
    return jlog

### Build Cython code

If not done before, build the cython code on the command line.

In [2]:
! python setup.py build_ext --inplace

running build_ext


### Import simulation class

In [3]:
from trainGD import trainGD

### Parameters

In [4]:
# Data dimension
d = 100
# Number of data points
n = 40
# Choose p 
p = 10
# Number of S random instances
nS = 10
# Signal-to-noise ratio
snr = 1./5.
# Max number of (S)GD iterations
n_it_max= int(1e6)  
##### Number of SGD runs 
n_SGD = 1
##### Store only the terminal point in the dynamics
only_end = False
##### Learning rate
lr = 1./d
##### Data matrix
X = np.random.randn(n,d)
##### Beta star d-dimension (ground true)
beta_star_ = np.random.randn(d)
beta_star = np.copy(beta_star_) / np.linalg.norm(beta_star_)
###### Initializing the estimator
beta0_ = np.random.randn(d)
beta0  = np.copy(beta0_)/np.linalg.norm(beta0_)
##### Targets
y = X @ beta_star + snr*np.random.randn(n)
#### Plot
log_x_max = (np.log10(n_it_max)-1).astype(int)  
plot_list = xlog_scale(log_x_max, scale=1., log_base= 10)
if only_end:
    plot_list = plot_list[-1:]
plot_list = np.insert(plot_list, 0, 0)

In [5]:
print(plot_list)
print(plot_list.shape)

[      0       1       2       3       4       5       6       7       8
       9      10      20      30      40      50      60      70      80
      90     100     200     300     400     500     600     700     800
     900    1000    2000    3000    4000    5000    6000    7000    8000
    9000   10000   20000   30000   40000   50000   60000   70000   80000
   90000  100000  200000  300000  400000  500000  600000  700000  800000
  900000 1000000]
(56,)


### Training

In [6]:
betaGD = trainGD(p=p, nS=nS, lr=lr, X=X, y=y, beta0= beta0, plot_list=plot_list, n=n, d=d, n_it_max=n_it_max)

Beginning GD training ----- p= 10 --- nS= 10
Time elapsed GD: 22.803155
End ----- p= 10 --- nS= 10
 


In [7]:
betaGD

array([[ 7.15469408e-02,  7.68967350e-02, -9.04543553e-02,
        -7.75515762e-02, -5.97368481e-03,  1.29601259e-01,
         2.85261241e-03,  1.71508410e-01, -1.47512991e-02,
         9.45496437e-02],
       [ 7.09757381e-02,  7.73753408e-02, -9.05141754e-02,
        -7.73551786e-02, -5.96403381e-03,  1.28733396e-01,
         4.85564792e-03,  1.69585839e-01, -1.45177472e-02,
         9.32283193e-02],
       [ 7.04119765e-02,  7.78492869e-02, -9.05678523e-02,
        -7.71573320e-02, -5.95897666e-03,  1.27873559e-01,
         6.83317921e-03,  1.67684389e-01, -1.42865516e-02,
         9.19178856e-02],
       [ 6.98555746e-02,  7.83186257e-02, -9.06155199e-02,
        -7.69580739e-02, -5.95843050e-03,  1.27021652e-01,
         8.78553719e-03,  1.65803830e-01, -1.40576863e-02,
         9.06182273e-02],
       [ 6.93064520e-02,  7.87834088e-02, -9.06573097e-02,
        -7.67574413e-02, -5.96231370e-03,  1.26177579e-01,
         1.07130484e-02,  1.63943930e-01, -1.38311258e-02,
         8.

In [8]:
print(betaGD.shape)

(56, 10)


In [9]:
with open('results/'+'betahowto.npy', 'wb') as f:
    np.save(f, betaGD)   

with open('results/'+'data/'+'plotlist.npy', 'wb') as f:
         np.save(f, plot_list)

with open('results/'+'data/'+ 'beta_starhowto.npy', 'wb') as f:
    np.save(f, beta_star)  

#### The set of random instances S

By construction, the set of S random instances is constructed inside the '.pyx' as follows:

In [10]:
np.random.seed(nS)
S  = np.random.choice(d, size=p, replace=False)
with open('results/'+'data/'+ 'S.npy', 'wb') as f:
    np.save(f, S)  

### In the paper

To get the training weights used in the paper, use the script `scrGD.py` after building the Cython code. One might want to appropriately set `intI` and `intF` to get a particular interval in `alpha`. 

In [ ]:
! python scrGD.py